In [2]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn import grid_search, linear_model as lm, cross_validation as cv, tree, preprocessing as prep

iowa = pd.read_csv('IowaLiquor.csv',parse_dates=[0])
iowa.columns = ['Date','StoreNo','City','Zip','CountyNo','County','CategoryNo','Category','VendorNo','ItemNo','Item','Volume','Cost','Retail','Quantity','TotSales','TotVolume','TotVolumeGals']
iowa

,Date,StoreNo,City,Zip,CountyNo,County,CategoryNo,Category,VendorNo,ItemNo,Item,Volume,Cost,Retail,Quantity,TotSales,TotVolume,TotVolumeGals
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.00,2.38
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.50,0.40
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.00,6.34
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.50,2.77
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.00,5.55
5,2015-04-20,2569,CEDAR RAPIDS,52402,57.0,Linn,1041100.0,AMERICAN DRY GINS,205,31473,New Amsterdam Gin,1750,$13.32,$19.98,6,$119.88,10.50,2.77
6,2015-08-05,2596,OTTUMWA,52501,90.0,Wapello,1051010.0,AMERICAN GRAPE BRANDIES,85,52806,Korbel Brandy,750,$6.66,$9.99,3,$29.97,2.25,0.59
7,2015-06-25,3456,CLEAR LAKE,50428,17.0,Cerro Gordo,1012100.0,CANADIAN WHISKIES,65,10628,Canadian Club Whisky,1750,$15.75,$23.63,2,$47.26,3.50,0.92
8,2016-01-04,4757,BONDURANT,50035,77.0,Polk,1032080.0,IMPORTED VODKA,370,34006,Absolut Swedish Vodka 80 Prf,750,$11.49,$17.24,4,$68.96,3.00,0.79
9,2015-11-10,4346,SHELLSBURG,52332,6.0,Benton,1081315.0,CINNAMON SCHNAPPS,65,82610,Dekuyper Hot Damn!,1000,$7.62,$11.43,2,$22.86,2.00,0.53


In [3]:
#Change dollar columns to a number
iowa['TotSales'] = iowa[['TotSales']].replace('[\$,]','',regex=True).astype(float)
iowa['Retail'] = iowa[['Retail']].replace('[\$,]','',regex=True).astype(float)
iowa['Cost'] = iowa[['Cost']].replace('[\$,]','',regex=True).astype(float)

In [4]:
import datetime as dt
#Add the Year variable
iowa['Year'] = pd.DatetimeIndex(iowa['Date']).year
#Quarter
iowa['Quarter'] = pd.DatetimeIndex(iowa['Date']).quarter
#TotalMargin
iowa['TotMargin'] = (iowa['Retail']-iowa['Cost'])*iowa['Quantity']
#PricePerLitre
iowa['PricePerLitre'] = iowa['TotSales']/iowa['Quantity']

In [5]:
daterange = lambda x:np.max(x)-np.min(x)

def aggsales(year=0,quarter=0):
    
    if year != 0 and quarter!=0:
        df = iowa[(iowa['Year']==year) & (iowa['Quarter']==quarter)]
    else:
        df = iowa
    
    dfout = pd.pivot_table(df,
                   index=['Quarter','StoreNo'],
                   values=['TotSales',
                           'Date',
                           'Quantity',
                           'TotMargin',
                           'TotVolume'
                          ],
                   aggfunc = {'TotSales':{'TotSales':np.sum},
                              'TotMargin':{'TotMargin':np.sum},
                              'TotVolume':{'TotVolume':np.sum},
                              'Quantity':{'TotQty':np.sum,'TotNoSales':len},
                              'Date':{'DateRange':daterange},
                             })
    
    dfout.columns = dfout.columns.droplevel()
    dfout['AvgBottlePrice'] = dfout['TotSales']/dfout['TotQty']
    dfout['AvgSaleAmount'] = dfout['TotSales']/dfout['TotNoSales']
    dfout['AvgPricePerLitre'] = dfout['TotSales']/dfout['TotQty']
    dfout['AvgMarginPercent'] = dfout['TotMargin']/dfout['TotSales']
    
    return dfout

sales_2015q1 = aggsales(2015,1)
sales_2016q1 = aggsales(2016,1)
sales_agg = aggsales()
sales_2015q1

DateRange  TotMargin  TotNoSales  TotQty  TotVolume  \
Quarter StoreNo                                                        
1       2106       77 days   13108.37         129    2705    2526.10   
        2113       77 days     944.72          42     196     177.11   
        2130       77 days    8108.68          87    1533    1447.25   
        2152       77 days     668.60          32     154     151.74   
        2178       77 days    1961.28          48     490     409.81   
        2190       84 days    9828.30         347    2557    1666.58   
        2191       84 days    9723.40         151    1868    1957.28   
        2200       77 days    1641.80          84     338     367.72   
        2205       84 days    2138.97          70     466     375.38   
        2228       84 days    1736.43          60     372     405.62   
        2233       84 days    1796.01          43     368     419.85   
        2238       70 days     286.28           7      49      42.00   
        2248       77 days    4227.24         101     665     649.54   
        2285       77 days    3345.90          58     522     503.45   
        2290       77 days    4083.72         160     870     817.58   
        2327       84 days     751.31          43     174     183.27   
        2353       84 days    5946.70          99    1282    1645.40   
        2367       84 days     674.80          22     175     158.00   
        2413       83 days    7501.02         113    1511    1538.37   
        2445       84 days     405.69          32     107      95.11   
        2448       84 days    1887.26         112     361     365.02   
        2459       77 days     538.07          22     133     137.00   
        2460       84 days    2256.58          82     586     575.26   
        2465       83 days    1875.51          98     452     400.12   
        2475       84 days    1891.99          20     394     654.15   
        2478       84 days    1486.90          19     224     210.00   
        2487       84 days    2427.71         120     500     510.89   
        2498       84 days      93.13          10      40      27.87   
        2500       77 days   13435.42         282    2950    3160.76   
        2501       77 days   10502.76         321    2321    2168.65   
...                    ...        ...         ...     ...        ...   
        5086       42 days     270.50           8     142      59.70   
        5088       70 days     159.78           4      36      30.75   
        5089       77 days     294.74          33     103      55.10   
        5090       73 days     735.53          20     193     132.59   
        5091       10 days     178.72          11      42      36.88   
        5092       63 days     605.58          13     168      99.00   
        5093       63 days     811.13          56     232     161.92   
        5094       56 days     172.14           6      87      34.72   
        5096       28 days     683.46          24     126     140.50   
        5097       56 days    1056.36          25     216     210.00   
        5098       46 days     453.97          22     178      68.45   
        5099       40 days     417.05          16     109      70.31   
        5100       42 days     727.56          14     336     111.90   
        5101       42 days     567.27          12     133      69.95   
        5102       53 days    9237.52         133    1532    1451.75   
        5103        0 days      14.26           1       2       1.50   
        5104        0 days     520.89          49      97      97.78   
        5105       21 days     911.14          34     230     211.32   
        5106       21 days     907.68          14     204     219.00   
        5108       28 days     333.38          14     107      63.75   
        5112       28 days     268.02           4      50      46.50   
        5113       21 days     809.76          16     314     116.40   
        5114        0 days     135.39           6      30

In [ ]:
treeReg = tree.DecisionTreeRegressor()
linReg = lm.LinearRegression()
TSReg = lm.TheilSenRegressor()
RANReg = lm.RANSACRegressor()

In [ ]:
clf = grid_search.GridSearchCV(ols,{'max_depth':range(1,10)})

clf.fit(train_X,train_Y)
clf.best_estimator_.max_depth